In [1]:
!pip install rasterio
!pip install ipygee
!pip install tslearn
!pip install earthengine-api
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.6 MB/s eta 0:00:00
  Created wheel for ipygee: filename=ipygee-0.0.18-py3-none-any.whl size=41204 sha256=6bf4666a4ade07fc07c1e840f7f87cef6fa6782093cd2cf58f373475d6d14269
  Stored in directory: /root/.cache/pip/wheels/6c/50/ac/d40610837fb4e308655017ff83aa14f9cb45adcbbc5eb501f0
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92089 sha256=ff5020122d4d7e9b0e4f3529ee2a5e549232f9f12d2f55ca0fce3b171a0c64c2
  Stored in directory: /root/.cache/pip/wheels/e9/a8/7f/40bf5aafcc334b20cd12d0328aff7861e8ea655c20190967fb
Successfully built ipygee geetools
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-alangtz51')

In [3]:
import folium
from folium import plugins
from IPython.display import Image
import geopandas as gpd
import json
print(folium.__version__)
from ipygee import*
import math
import pandas as pd
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

0.14.0


In [4]:
AOI =  ee.Geometry.Polygon(
[[[-111.13282620813519, 31.294142720019394],
[-111.13282620813519, 30.687451376881327],
[-110.49250469198265, 30.687451376881327],
[-110.49250469198265,31.294142720019394],
[-111.13282620813519, 31.294142720019394]]])

#[[[-56.37397887990624, -12.737207954893526],
#[-56.37397887990624, -13.300834158918619],
#[-55.37113311574608, -13.300834158918619],
#[-55.37113311574608, -12.737207954893526]]])

points = ee.FeatureCollection.randomPoints(AOI,100)


In [5]:
months = ee.List.sequence(1,12)
years = ee.List.sequence(2016, 2019)

In [6]:
MD_NDVI = ee.ImageCollection('MODIS/MOD09GA_006_NDVI') \
    .filterDate('2016-01-01', '2019-12-31') \
    .filterBounds(AOI) \
    .select('NDVI')

In [7]:
modis_ndvi = MD_NDVI.median().clip(AOI)
mean_ndvi = MD_NDVI.mean().clip(AOI)

In [8]:
vis_params = {'min': 0, 'max': 1, 'palette':
['red', 'yellow','green']}
basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}
def add_ee_layer(self, ee_object, vis_params, name):
 try:
  if isinstance(ee_object, ee.image.Image):
   map_id_dict = ee.Image(ee_object).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
   ee_object_new = ee_object.mosaic()
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.geometry.Geometry):
   folium.GeoJson(
   data = ee_object.getInfo(),
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
   ee_object_new = ee.Image().paint(ee_object, 0, 2)
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

 except:
  print("Could not display {}".format(name))
folium.Map.add_ee_layer = add_ee_layer
#-13.0912068,-55.9881647
my_map = folium.Map(location=[31.3099,-110.6363],
                    zoom_start=10)
basemaps['Google Maps'].add_to(my_map)
my_map.add_ee_layer(modis_ndvi, vis_params, 'NDVI')
my_map.add_ee_layer(points.geometry(), {}, 'Points')
my_map.add_child(folium.LayerControl())
display(my_map)

In [9]:
def monthly(collection):
    img_coll = ee.ImageCollection([])
    for y in years.getInfo():
        for m in months.getInfo():
            filtered = collection.filter(ee.Filter.calendarRange(y, y, 'year')) \
                                 .filter(ee.Filter.calendarRange(m, m, 'month'))
            filtered = filtered.median()
            img_coll = img_coll.merge(filtered.set('year', y)
                                              .set('month', m)
                                              .set('system:time_start',
                                                   ee.Date.fromYMD(y, m, 1)
                                                   .getInfo()['value']))
    return img_coll

Monthly_MD = monthly(MD_NDVI)


In [10]:
#Point_1 = ee.Geometry.Point([-56.1070717726807, -13.168187045136754])
Point_1 = ee.Geometry.Point([-110.83625624275916, 31.017276970052947])

MD_ndvi = chart.Image.series(
    **{
        'imageCollection': Monthly_MD,
        'region': Point_1,
        'reducer': ee.Reducer.mean(),  # Añadir coma después de ee.Reducer.mean()
        'scale': 250,
        'xProperty': 'system:time_start'
    }
)

MD_ndvi.renderWidget(width='100%')


HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [11]:
dependent = 'NDVI'
harmonics = 3
harmonicFrequencies = list(range(1, harmonics + 1))

def getNames(base, lst_freq):
    name_lst = []
    for i in lst_freq:
        name_lst.append(ee.String(base + str(i)))
    return name_lst

cosNames = getNames('cos_', harmonicFrequencies)
sinNames = getNames('sin_', harmonicFrequencies)

# Corrección en la definición de independents
independents = ee.List(['constant', 't']).cat(cosNames).cat(sinNames)

In [12]:
def addConstant (image) :
 return image.addBands(ee.Image(1));

def addTime (image) :
 date = ee.Date(image.get('system:time_start'));
 years = date.difference(ee.Date('1970-01-01'), 'year');
 timeRadians = ee.Image(years.multiply(2 * math.pi));
 return image.addBands(timeRadians.rename('t').float());

def addHarmonics (image) :
 frequencies = ee.Image.constant(harmonicFrequencies)
 time = ee.Image(image).select('t')
 cosines = time.multiply(frequencies).cos().rename(cosNames)
 sines = time.multiply(frequencies).sin().rename(sinNames)
 return image.addBands(cosines).addBands(sines)

In [13]:
import math  # Asegúrate de importar el módulo math

# Define las funciones addConstant, addTime y addHarmonics aquí

# Asegúrate de que las variables harmonicFrequencies, cosNames y sinNames estén definidas o pasadas correctamente

# Resto del código
harmonicMODIS2 = Monthly_MD.map(addConstant) \
                           .map(addTime) \
                           .map(addHarmonics)

harmonicTrend = harmonicMODIS2.select(independents.add(dependent)) \
                               .reduce(ee.Reducer.linearRegression(independents.length(), 1))

harmonicTrendCoefficients = harmonicTrend.select('coefficients') \
                                         .arrayProject([0]) \
                                         .arrayFlatten([independents])

fittedHarmonic = harmonicMODIS2.map(lambda image: image.addBands(image.select(independents)
                                                                  .multiply(harmonicTrendCoefficients)
                                                                  .reduce('sum')
                                                                  .rename('fitted')))


In [14]:
MD_ndvi = chart.Image.series(**{'imageCollection': fittedHarmonic,  'region': Point_1,
'reducer': ee.Reducer.mean(),
'bands' : 'fitted',
'scale': 250,
'xProperty': 'system:time_start'})
MD_ndvi.renderWidget(width='100%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…